<a href="https://colab.research.google.com/github/TanushGoel/Machine-Learning-Playground/blob/master/MusicAI_1_0_LSTM_Nocturnes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
from music21 import converter, instrument, note, chord, stream, common
import os
import os.path
from os import path
import zipfile
import glob
import pickle
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Concatenate
from keras.layers.core import*
from keras import initializers
from keras.callbacks import ModelCheckpoint
from google.colab import files

Using TensorFlow backend.


In [0]:
# upload zip file first
for i in os.listdir():
  global zip_file, folder
  if zipfile.is_zipfile(i):
      zip_file = str(i)
      folder = zip_file[:-4]
      with zipfile.ZipFile(zip_file, 'r') as zip_ref:
          zip_ref.extractall(os.mkdir(folder))

In [0]:
# one file at a time
notes = []
count = 0
total = len(os.listdir(folder))

for file in glob.glob(folder+"/*.mid"):

    count+=1
    print(f"{count*100/total:1.2f}% Complete")

    try:  
      midi = converter.parse(file)
      parts = instrument.partitionByInstrument(midi)

      if parts: # file has instrument parts
          if len(parts.parts) > 1: # the file has more than one instrument
            print(file, "has more than one instrument")
            continue
          else:
            notes_to_parse = parts.parts[0].recurse()
      else: # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))
  
    except:
      print(file, "could not be parsed")

0.55% Complete
1.09% Complete
1.64% Complete
2.19% Complete
2.73% Complete
3.28% Complete
3.83% Complete
4.37% Complete
4.92% Complete
5.46% Complete
6.01% Complete
6.56% Complete
7.10% Complete
7.65% Complete
8.20% Complete
8.74% Complete
9.29% Complete
9.84% Complete
10.38% Complete
10.93% Complete
11.48% Complete
12.02% Complete
12.57% Complete
13.11% Complete
13.66% Complete
14.21% Complete
14.75% Complete
15.30% Complete
15.85% Complete
16.39% Complete
16.94% Complete
17.49% Complete
18.03% Complete
18.58% Complete
19.13% Complete
19.67% Complete
20.22% Complete
20.77% Complete
21.31% Complete
21.86% Complete
22.40% Complete
22.95% Complete
23.50% Complete
24.04% Complete
24.59% Complete
25.14% Complete
25.68% Complete
26.23% Complete
26.78% Complete
27.32% Complete
27.87% Complete
28.42% Complete
28.96% Complete
29.51% Complete
30.05% Complete
30.60% Complete
31.15% Complete
31.69% Complete
32.24% Complete
32.79% Complete
33.33% Complete
33.88% Complete
34.43% Complete
34.97% Com

In [0]:
# multiple files in parallel
if not path.isdir("data"):
  os.mkdir("data")

filez = []
for file in glob.glob(folder+"/*.mid"):
    filez.append(file)

def get_notes(file):

  notes = []
  notes_to_parse = None
  midi = converter.parse(file)
  parts = instrument.partitionByInstrument(midi)

  try:
    if parts: # file has instrument parts
        if len(parts.parts) > 1: # the file has more than one instrument
          print(file, "has more than one instrument")
          return
        else:
          notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
  except:
    print(file, "could not be parsed")
    
  with open('data/notes', 'wb') as filepath:
    pickle.dump(notes, filepath)

  return notes

output = common.runParallel(filez, parallelFunction=get_notes)

notes = []
with (open("data/notes", "rb")) as openfile:
    while True:
        try:
          notes.append(pickle.load(openfile))
        except EOFError:
          print("EOFError")
          break

notes = []
if os.path.getsize("data/notes") > 0:      
    with open("data/notes", "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        notes = unpickler.load()

del filez

In [0]:
sequence_length = 125 # CHANGE - EXPERIMENT

# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    
n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

n_vocab = np.amax(network_input)

In [0]:
# normalize input
network_input = np.divide(network_input, float(n_vocab))
network_output = np_utils.to_categorical(network_output)

In [0]:
# LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True, kernel_initializer=initializers.RandomNormal(stddev=0.175),
    bias_initializer=initializers.Zeros()))
model.add(Dropout(0.225))
model.add(LSTM(512, return_sequences=True, kernel_initializer=initializers.RandomNormal(stddev=0.175),
    bias_initializer=initializers.Zeros()))
model.add(Dropout(0.225))
model.add(LSTM(256, kernel_initializer=initializers.RandomNormal(stddev=0.175),
    bias_initializer=initializers.Zeros()))
model.add(Dense(256, kernel_initializer=initializers.RandomNormal(stddev=0.175),
    bias_initializer=initializers.Zeros()))
model.add(Dropout(0.225))
model.add(Dense(n_vocab+1, kernel_initializer=initializers.RandomNormal(stddev=0.25),
    bias_initializer=initializers.Zeros()))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)              

In [0]:
num_notes = len(notes)
batch = 25
step = int(np.ceil(num_notes / float(batch)))
input_dim = int(network_input.shape[1])
hidden = int(network_input.shape[2])

In [0]:
# The LSTM  model -  output_shape = (batch, step, hidden)
model1 = Sequential()
model1.add(LSTM(input_dim=input_dim, output_dim=hidden, input_length=step, return_sequences=True))

# The weight model  - actual output shape  = (batch, step)
# after reshape : output_shape = (batch, step,  hidden)
model2 = Sequential()
model2.add(Dense(input_dim=input_dim, output_dim=step))
model2.add(Activation('softmax')) # Learn a probability distribution over each  step.
#Reshape to match LSTM's output shape, so that we can do element-wise multiplication.
model2.add(RepeatVector(hidden))
print(model2.output_shape)
model2.add(Permute(2, 1), input_shape=(1280, 1))

# The final model which gives the weighted sum:
model = Sequential()
merged = Concatenate()([model1, model2])
model.add(merged)  # Multiply each element with corresponding weight a[i][j][k] * b[i][j]
model.add(TimeDistributedMerge('sum')) # Sum the weighted elements.

#model.compile(loss='mse', optimizer='adam')

(None, 1, 1280)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(1280, 100..., units=1)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1280)`
  


TypeError: ignored

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [0]:
# checkpointer
checkpoint = ModelCheckpoint("MusicAI_best.hdf5", 
                             monitor='loss', 
                             verbose=1,        
                             save_best_only=True)

In [0]:
# train model
model.fit(network_input, network_output, epochs=50, batch_size=batch, callbacks=[checkpoint])

Epoch 1/50
31900/31900 [==============================] - 600s 19ms/step - loss: 4.5648 - acc: 0.0381

Epoch 00001: loss improved from inf to 4.56478, saving model to MusicAI_best.hdf5
Epoch 2/50
31900/31900 [==============================] - 581s 18ms/step - loss: 4.3164 - acc: 0.0444

Epoch 00002: loss improved from 4.56478 to 4.31637, saving model to MusicAI_best.hdf5
Epoch 3/50
31900/31900 [==============================] - 563s 18ms/step - loss: 4.2842 - acc: 0.0467

Epoch 00003: loss improved from 4.31637 to 4.28423, saving model to MusicAI_best.hdf5
Epoch 4/50
31900/31900 [==============================] - 563s 18ms/step - loss: 4.2670 - acc: 0.0474

Epoch 00004: loss improved from 4.28423 to 4.26698, saving model to MusicAI_best.hdf5
Epoch 5/50
31900/31900 [==============================] - 562s 18ms/step - loss: 4.2294 - acc: 0.0508

Epoch 00005: loss improved from 4.26698 to 4.22944, saving model to MusicAI_best.hdf5
Epoch 6/50
31900/31900 [==============================] - 5

In [0]:
# make piece
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []

# generate 10000 notes
for note_index in range(10000):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]

In [0]:
# make piece
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5 # offset += 0.25

In [0]:
# stream piece into midi file
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output.mid')

'output.mid'

In [0]:
# download piece
files.download('/content/output.mid')